In [ ]:
#| default_exp callbacks.evaluation

# Evaluation

> Simple callbacks to evaluate the current training model on a dataset at different training times.

In [ ]:
#| export
from typing import Dict

import tensorflow as tf
from tensorflow.keras.callbacks import Callback

We're going to build a class that stores a given dataset and calls evaluate on the training model when needed to obtain the evaluation metrics. We want it to be flexible in a way that we can specify a number of epochs or batches as the evaluation frequency. This could be solved by having a different callback for epoch and batches, but we probably can get away with using only one.

In [ ]:
#| export
class EvaluateDataset(Callback):
    """Evaluates a given `tf.data.Dataset` at different training times."""

    def __init__(self,
                 dataset, # Dataset to be evaluated.
                 freq_epochs, # Number of epochs to wait between evaluations. `None` means not evaluating at an epoch interval.
                 freq_batches, # Number of batches to wait between evaluations. `None` means not evaluating at a batch interval.
                 ):
        self.dataset = dataset if isinstance(dataset, tf.data.Dataset) else self._convert_to_dataset(dataset)
        self.freq_epochs = freq_epochs
        self.freq_batches = freq_batches
        self.batches_seen, self.epochs_seen = 0, 0
        self._results_batches, self._results_epochs = [], []

    def _convert_to_dataset(self,
                            dataset, # Dataset to be converted.
                            ):
        """Tries to convert a dataset into a `tf.data.Dataset`."""
        return dataset

    def evaluate(self,
                 ) -> Dict: # Dictionary of evaluation results.
        """Calls the `.evaluate()` method of the given `model` on the stored `dataset`."""
        return self.model.evaluate(self.dataset)

    def on_train_batch_end(self,
                           batch, # Batch number in an epoch
                           logs=None, # Training logs.
                           ):
        if self.freq_batches is None: return
        else:
            if self.batches_seen % self.freq_batches == 0: 
                results = self.evaluate()
                self._results_batches.append(results)
            self.batches_seen += 1
    
    def on_epoch_end(self,
                     batch, # Batch number in an epoch
                     logs=None, # Training logs.
                     ):
        if self.freq_epochs is None: return
        else:
            if self.epochs_seen % self.freq_epochs == 0: 
                results = self.evaluate()
                self._results_epochs.append(results)
            self.epochs_seen += 1
    
    def _unpack_list_dicts(list_of_dicts):
        """Unpacks a list of dicts sharing keys into a dict with lists as values."""
        res = {}
        for result in list_of_dicts:
            for metric, value in result.items():
                res[metric].append(value)
        return res

    @property
    def results_batches(self):
        if len(self._results_batches) == 0: raise ValueError("No values stored yet.")
        return self._unpack_list_dicts(self._results_batches)

    @property
    def results_epochs(self):
        if len(self._results_epochs) == 0: raise ValueError("No values stored yet.")
        return self._unpack_list_dicts(self._results_epochs)